In [ ]:
!pip install tensorflow

In [2]:
import numpy as np
import pandas as pd

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import tensorflow as tf
from keras.datasets import fashion_mnist

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [4]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Expand dims to add channel: (28,28) → (28,28,1)
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# Resize to (150, 150) and convert to RGB (3 channels)
x_train = tf.image.resize(x_train, [150, 150])
x_test = tf.image.resize(x_test, [150, 150])

# Convert to RGB by duplicating channels (1 → 3)
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0



In [ ]:
input_shape = (150, 150, 3)

num_classes = 10



# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)


base_model_vgg = VGG19(include_top=False, input_shape=input_shape, pooling=None,weights="imagenet")
base_model_densenet = DenseNet201(include_top=False, input_shape=input_shape, pooling=None,weights="imagenet")

# Adding custom classification layers
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
#x = Dropout(0.5)(x)  # Dropout for regularization
x = Dense(200, activation="relu")(x)  # Use softmax for multi-class classification

# Final model
model = Model(inputs=base_model_vgg.input, outputs=x)

model.predict(x_train)